In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

<b> Getting plain html from wiki page and viewing as the nested html version </b>

In [2]:
# get plain html of wiki needs scraping
scrape_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_url = requests.get(scrape_url).text

# view as nested version 
soup = BeautifulSoup(html_url,'lxml')
# print(soup.prettify())

<b> trying to find a tag to separate table </b>

In [3]:
# separate table
table = soup.find('table',{'class':'wikitable sortable'})

<b> trying to find a tag to separate fields </b>

In [4]:
# separate fields
table_parts = table.find_all('td')

<b> initiating empty dataframe with column names, 
 and looping over the html table, and appending 1 row at a time </b>

In [5]:
# initiate dataframe
col_names = ['Postcode','Borough','Neighborhood']
df = pd.DataFrame(columns=col_names)

In [6]:
# loop over separated html and append to df row by row

for i in range(0,len(table_parts),3):
    
    pc, boro, neigh = table_parts[i].text.strip(), table_parts[i+1].text.strip(), table_parts[i+2].text.strip()
    
    df = df.append({'Postcode':pc,
                   'Borough':boro,
                   'Neighborhood':neigh},ignore_index=True)
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<b> removing rows where borough is not assigned and 
 simply grouping by postcode and borough and aggregating 
 neighborhood by comma </b>

In [7]:
# removing rows where borough = not assigned
df['Borough'].replace(to_replace='Not assigned', value = np.nan ,inplace=True)
df.dropna(inplace=True)

# group by postcode and borough and join Neighborhood by ','
dft = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
dft.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<b>Only 1 row has neighborhood as not assigned, simply replacing that with the Borough name</b>

In [8]:
dft['Neighborhood'].replace(to_replace='Neighborhood',value="Queen's Park",inplace=True)

In [9]:
dft.shape

(103, 3)

In [11]:
# creating a df to join to the main df 
postcode_lat_lon = 'https://cocl.us/Geospatial_data'

!wget -q -O 'toronto_m.geospatial_data.csv' postcodes_lat_lon

postcodes_data = pd.read_csv(postcode_lat_lon)
postcodes_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:

df2 = pd.merge(dft,postcodes_data,how='left',left_on='Postcode',right_on='Postal Code').drop(labels='Postal Code',axis=1)